### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- get Global Human Settlement Layer population estimates data using the Google Earth Engine functionalities
- clip it to an AOI

### Prerequisites
- earth engine api (ee) https://developers.google.com/earth-engine/apidocs
- gee map (gee), https://geemap.org/ by Qiusheng Wu
- create an account and a Google Cloud project in Google Developer, see here: https://developers.google.com/earth-engine/guides/auth

# init

In [ ]:
import os
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import logging

# functions

In [ ]:
# set up logging
logging.basicConfig(level=logging.INFO)

# define a function that exports the image to Google Drive
def export_image(image, filename, roi): #crs, projection, removed these arguments as they caused a resampling error
    try: 
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=filename,
            folder='gee_exports',
            region=roi,
            #crs=crs,
            #crsTransform=projection['transform'],
            maxPixels=1e13,
            fileFormat='GeoTIFF',
            formatOptions={
                'cloudOptimized': False
            }
        )
        task.start()
        logging.info(f'Exporting {filename} to Google Drive')
        return task
    except Exception as e:
        logging.error(f'An error has occurred: {e}')
        return None

def gee_to_tiff(input_layer, out_folder, output_filename, crs, roi):
    #projection = input_layer.projection().getInfo()
    
    try:
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        
        # Export a GEE layer to raster Gtiff
        geemap.ee_export_image_to_drive(
            input_layer,
            description=output_filename,
            folder=out_folder,
            maxPixels=1e13, #ressamples to a low resolution
            region = roi,
            crs = crs, #projection['crs'],
            fileFormat = 'GeoTIFF',
            scale = 10           
        )
        #raster = 
        #return raster
    except Exception as e:
        print("An error has occurred:", e)
    
# check image stats before export
def check_image_stats(image, roi): 
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=250,
        maxPixels=1e9       
    )
    return stats.getInfo()


# extract coordinates from the bounding box
def get_coords(gdf):
    coords = gdf.envelope #.to_crs(epsg='4326') the CRS is defined at the import
    epsg_coords = coords.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    coi = roi.centroid(maxError=1)
    return epsg_coords, roi, rec_roi, coi


In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='') # Replace with your GEE project #'ee-cmirb-her'
print(ee.__version__)

In [ ]:
# check if the authentication is working
print(ee.Image("JRC/GHSL/P2023A/GHS_POP/1975").getInfo()) #.get("smod_code")

# imports

In [ ]:
#input a vector and a raster file
AOI_path = Path('../data/processed/')

AOI_file_JAK = 'JAK_LIM_BoundingBox_AOI_A.shp' #Jakarta
AOI_file_MUM = 'MUM_LIM_BoundingBox_AOI_A.shp' #Mumbai
AOI_file_MAN = 'MAN_LIM_BoundingBox_AOI_A.shp' #Manila

#read the vector files
AOI_gdf_JAK = gpd.read_file(AOI_path/AOI_file_JAK).to_crs(epsg=4326)
AOI_gdf_MUM = gpd.read_file(AOI_path/AOI_file_MUM).to_crs(epsg=4326)
AOI_gdf_MAN = gpd.read_file(AOI_path/AOI_file_MAN).to_crs(epsg=4326)

# get the coordinates
JAK_aoi_crs, JAK_roi, JAK_rec_roi, JAK_coi = get_coords(AOI_gdf_JAK)
MUM_aoi_crs, MUM_roi, MUM_rec_roi, MUM_coi = get_coords(AOI_gdf_MUM)
MAN_aoi_crs, MAN_roi, MAN_rec_roi, MAN_coi = get_coords(AOI_gdf_MAN)

#print(coords)
print('The CRS are:', MUM_aoi_crs, JAK_aoi_crs,  MAN_aoi_crs)

# Get

In [ ]:
# defining EE collection
# data from JRC GHSL https://human-settlement.emergency.copernicus.eu/ghs_pop2023.php
# earth engine collections from the JRC:
# Degree of urbanisation 1975-2023 ee.ImageCollection("JRC/GHSL/P2023A/GHS_SMOD")
# Global Built Up 1975-2023 ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")
# code based on https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP


#baseChange = [{featureType: 'all', stylers: [{saturation: -100}, {lightness: 45}]}]
#Map.setOptions('baseChange', {'baseChange': baseChange});
image1985 = ee.Image('JRC/GHSL/P2023A/GHS_POP/1985')
image2000 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2000')
image2015 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2015')
print('1985 stats:', check_image_stats(image1985, MUM_roi))
print('2000 stats:', check_image_stats(image2000, MUM_roi))
print('2015 stats:', check_image_stats(image2015, MUM_roi))


In [ ]:
Map = geemap.Map()
Map.centerObject(MUM_coi, zoom=9)

populationCountVis = {
  'min': 0.0,
  'max': 100.0,
  'palette':['000004', '320A5A', '781B6C', 'BB3654', 'EC6824', 'FBB41A', 'FCFFA4']
}
image1985 = image1985.updateMask(image1985.gt(0))
image2000 = image2000.updateMask(image2000.gt(0))
image2015 = image2015.updateMask(image2015.gt(0))
Map.addLayer(image1985, populationCountVis, 'Population count, 1985')
Map.addLayer(image2000, populationCountVis, 'Population count, 1990')
Map.addLayer(image2015, populationCountVis, 'Population count, 2015')
Map.addLayer(MAN_rec_roi, {}, 'ROI', )
Map
print('1985 stats:', check_image_stats(image1985, MAN_roi))
print('2000 stats:', check_image_stats(image2000, MAN_roi))
print('2015 stats:', check_image_stats(image2015, MAN_roi))

In [ ]:
Map1985 = geemap.Map()
Map1985.centerObject(MUM_coi, zoom=9)

Map1985.addLayer(image1985, populationCountVis, 'Population count, 1985')
Map1985

# clip

In [ ]:
# create individual images for each of the AOIs

# JAK
JAK_pop2015 = image2015.clip(JAK_rec_roi)
JAK_pop2000 = image2000.clip(JAK_rec_roi)
JAK_pop1985 = image1985.clip(JAK_rec_roi)
JAK_crs='EPSG:'+str(JAK_aoi_crs)
# MUM
MUM_pop2015 = image2015.clip(MUM_rec_roi)
MUM_pop2000 = image2000.clip(MUM_rec_roi)
MUM_pop1985 = image1985.clip(MUM_rec_roi)
MUM_crs='EPSG:'+str(MUM_aoi_crs)
# MAN
MAN_pop2015 = image2015.clip(MAN_rec_roi)
MAN_pop2000 = image2000.clip(MAN_rec_roi)
MAN_pop1985 = image1985.clip(MAN_rec_roi)
MAN_crs='EPSG:'+str(MAN_aoi_crs)

# check stats for each image
print('JAK_pop2020 stats:', check_image_stats(JAK_pop2015, JAK_rec_roi))
print('JAK_pop2000 stats:', check_image_stats(JAK_pop2000, JAK_rec_roi))
print('JAK_pop1985 stats:', check_image_stats(JAK_pop1985, JAK_rec_roi))
print('MUM_pop2020 stats:', check_image_stats(MUM_pop2015, MUM_rec_roi))
print('MUM_pop2000 stats:', check_image_stats(MUM_pop2000, MUM_rec_roi))
print('MUM_pop1985 stats:', check_image_stats(MUM_pop1985, MUM_rec_roi))
print('MAN_pop2020 stats:', check_image_stats(MAN_pop2015, MAN_rec_roi))
print('MAN_pop2000 stats:', check_image_stats(MAN_pop2000, MAN_rec_roi))
print('MAN_pop1985 stats:', check_image_stats(MAN_pop1985, MAN_rec_roi))


In [ ]:
Map = geemap.Map()
Map.centerObject(MAN_coi, zoom=10)


Map.addLayer(MAN_pop2020, {'min': 0.5, 'max':1000 , 'palette': ['000004', '320A5A', '781B6C', 'BB3654', 'EC6824', 'FBB41A', 'FCFFA4']}, 'population_count', True)
# Map.addLayer(MUM_pop, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'population_count', True)
# Map.addLayer(MAN_pop, {'min': 0, 'max': 1, 'palette': ['0000ff', '003cd5']}, 'population_count', True)
# Map.addLayer(roi, {}, 'ROI')
Map

# export

In [ ]:
# export the images for Jakarta
# the function exports the images in the original CRS, as the reprojection caused a resampling error

jak_task_2015 = export_image(JAK_pop2015, 'JAK_POP_population_2015_GHSL', JAK_rec_roi) 
jak_task_2000 = export_image(JAK_pop2000, 'JAK_POP_population_2000_GHSL', JAK_rec_roi) 
jak_task_1985 = export_image(JAK_pop1985, 'JAK_POP_population_1985_GHSL', JAK_rec_roi) 

# check task status
tasks = [jak_task_2015, jak_task_2000, jak_task_1985]
for task in tasks: 
    if task: 
        print(f'Task {task.status()["id"]} is {task.status()["state"]}')
    else:
        print('Task failed')

In [ ]:
# export the images for Mumbai
# the function exports the images in the original CRS, as the reprojection caused a resampling error

mum_task_2015 = export_image(MUM_pop2015, 'MUM_POP_population_2015_GHSL', MUM_rec_roi) 
mum_task_2000 = export_image(MUM_pop2000, 'MUM_POP_population_2000_GHSL', MUM_rec_roi) 
mum_task_1985 = export_image(MUM_pop1985, 'MUM_POP_population_1985_GHSL', MUM_rec_roi)  

# check task status
tasks = [mum_task_2015, mum_task_2000, mum_task_1985]
for task in tasks: 
    if task: 
        print(f'Task {task.status()["id"]} is {task.status()["state"]}')
    else:
        print('Task failed')

In [ ]:
# export the images for Manila
# the function exports the images in the original CRS, as the reprojection caused a resampling error

man_task_2015 = export_image(MAN_pop2015, 'MAN_POP_population_2015_GHSL', MAN_rec_roi) 
man_task_2000 = export_image(MAN_pop2000, 'MAN_POP_population_2000_GHSL', MAN_rec_roi) 
man_task_1985 = export_image(MAN_pop1985, 'MAN_POP_population_1985_GHSL', MAN_rec_roi) 

# check task status
tasks = [man_task_2015, man_task_2000, man_task_1985]
for task in tasks: 
    if task: 
        print(f'Task {task.status()["id"]} is {task.status()["state"]}')
    else:
        print('Task failed')

In [ ]:
jak_task_2015 = export_image(JAK_pop2015, 'JAK_POP_population_2015_GHSL', JAK_rec_roi) 
mum_task_2015 = export_image(MUM_pop2015, 'MUM_POP_population_2015_GHSL', MUM_rec_roi) 
man_task_2015 = export_image(MAN_pop2015, 'MAN_POP_population_2015_GHSL', MAN_rec_roi) 

tasks = [jak_task_2015, mum_task_2015, man_task_2015]
for task in tasks: 
    if task: 
        print(f'Task {task.status()["id"]} is {task.status()["state"]}')
    else:
        print('Task failed')